In [1]:
# set working directory to same place ass app.py to import programs the same way as the app
import os
current_directory = os.getcwd()
if 'notebooks' in current_directory:
    parent_directory = os.path.abspath(os.path.join(current_directory, os.pardir))
    os.chdir(parent_directory)
os.getcwd()

'/Users/elisealstad/code/mybook-dashboard'

In [2]:
# Improt packages
import pandas as pd
import json 
import numpy as np

# Import functions from apps folder
# from apps.collect_data import *

pd.set_option('max_colwidth', 50)
pd.set_option('display.max_columns', 80)

In [3]:
mybooksgr = pd.read_csv("assets/goodreads_library_export.csv")
mybooksgr = mybooksgr.rename(columns=lambda x: x.replace(' ', '_'))
df = mybooksgr.head(5)

In [4]:
# used to be fast but not anymore

import requests
import pandas as pd
import numpy as np
import aiohttp
import asyncio
import nest_asyncio

async def get_book_info_async(session, book_name, author_name, api_key):
    base_url = 'https://www.googleapis.com/books/v1/volumes'
    params = {
        'q': f'intitle:{book_name}+inauthor:{author_name}',
        'key': api_key,
        'maxResults': 1,
        'fields': 'items(volumeInfo/title,volumeInfo/authors,volumeInfo/publishedDate,volumeInfo/description,volumeInfo/industryIdentifiers,volumeInfo/pageCount,volumeInfo/categories,volumeInfo/averageRating,volumeInfo/ratingsCount,volumeInfo/language)'
    }

    async with session.get(base_url, params=params) as response:
        if response.status == 200:
            data = await response.json()
            if 'items' in data and len(data['items']) > 0:
                return data['items'][0]['volumeInfo']
            print('request one done')
        else:
            print(f"Error {response.status}: {await response.text()}")
            return None

async def get_book_info(book_name, author_name, api_key):
    async with aiohttp.ClientSession() as session:
        return await get_book_info_async(session, book_name, author_name, api_key)

def book_info_add(df, api_key):
    async def get_book_info_wrapper(row):
        book_name = row['Title']
        author_name = row['Author']
        await asyncio.sleep(1)
        print(book_name)
        return await get_book_info(book_name, author_name, api_key)

    # Use nest_asyncio to allow running asyncio in a notebook
    nest_asyncio.apply()

    # Create an event loop
    loop = asyncio.get_event_loop()

    # Alternative 1 # 2.5  mins
    # Run the asynchronous code
    tasks = [get_book_info_wrapper(row) for _, row in df.iterrows()]
    book_infos = loop.run_until_complete(asyncio.gather(*tasks))

    # alternative 2 
    # # Introduce a delay of 0.1 seconds between requests
    # book_infos = asyncio.run(asyncio.gather(*[get_book_info_wrapper(row) for _, row in df.iterrows()]))
    
    
    ###########

    combined_book_info = pd.DataFrame()

    for book_info in book_infos:
        if book_info:
            print('one')
            authors = book_info.get('authors', [np.nan])
            publish_date = book_info.get('publishedDate', np.nan)
            description = book_info.get('description', np.nan)
            identifiers = book_info.get('industryIdentifiers', [])
            isbn = identifiers[0]['identifier'] if identifiers else np.nan
            page_count = book_info.get('pageCount', np.nan)
            categories = book_info.get('categories', [np.nan])
            average_rating = book_info.get('averageRating', np.nan)
            rating_count = book_info.get('ratingsCount', np.nan)
            language = book_info.get('language', np.nan)

            book_info_df = pd.DataFrame({
                'Title': [book_info.get('title', np.nan)],
                'Author(s)': [", ".join(map(str, authors))],
                'Publish Date': [publish_date],
                'Description': [description],
                'ISBN': [isbn],
                'Page Count': [page_count],
                'Categories': [", ".join(map(str, categories))],
                'Average Rating': [average_rating],
                'Rating Count': [rating_count],
                'Language': [language]
            })
            combined_book_info = pd.concat([combined_book_info, book_info_df])

    # Reset the index of the combined DataFrame
    combined_book_info = combined_book_info.reset_index(drop=True)
    combined_book_info = combined_book_info.rename(columns=lambda x: x.replace(' ', '_'))

    return combined_book_info
from apps.api import api_key
result = book_info_add(mybooksgr.tail(6), api_key)
print(result)



Pachinko
The Evening and the Morning (Kingsbridge, #0)
All the Light We Cannot See
1984
Monkey Hunting: A Novel (Ballantine Reader's Circle)
Animal Farm
one
one
one
one
one
one
                                     Title        Author(s) Publish_Date  \
0  Pachinko (National Book Award Finalist)      Min Jin Lee   2017-02-07   
1              The Evening and the Morning      Ken Follett   2020-09-15   
2              All the Light We Cannot See    Anthony Doerr   2023-10-03   
3                                     1984    George Orwell   2013-09-03   
4                           Monkey Hunting  Cristina García   2007-12-18   
5                              Animal Farm    George Orwell         2009   

                                         Description           ISBN  \
0  A New York Times Top Ten Book of the Year and ...  9781455563913   
1  #1 New York Times Bestseller An Amazon Best Bo...  9781984882028   
2  *COMING IN NOVEMBER AS A NETFLIX LIMITED SERIE...  9781668017340   
3  A P

In [5]:

import requests
import pandas as pd
import numpy as np
import aiohttp
import asyncio
import nest_asyncio
import time

async def get_book_info_async(session, book_name, author_name, api_key):
    base_url = 'https://www.googleapis.com/books/v1/volumes'
    params = {
        'q': f'intitle:{book_name}+inauthor:{author_name}',
        'key': api_key,
        'maxResults': 1,
        'fields': 'items(volumeInfo/title,volumeInfo/authors,volumeInfo/publishedDate,volumeInfo/description,volumeInfo/industryIdentifiers,volumeInfo/pageCount,volumeInfo/categories,volumeInfo/averageRating,volumeInfo/ratingsCount,volumeInfo/language)'
    }

    async with session.get(base_url, params=params) as response:
        if response.status == 200:
            data = await response.json()
            if 'items' in data and len(data['items']) > 0:
                return data['items'][0]['volumeInfo']
            print('request one done')
        else:
            print(f"Error {response.status}: {await response.text()}")
            return None

async def get_book_info(book_name, author_name, api_key):
    async with aiohttp.ClientSession() as session:
        return await get_book_info_async(session, book_name, author_name, api_key)

async def book_info_add(df, api_key):
    async def get_book_info_wrapper(row):
        book_name = row['Title']
        author_name = row['Author']
        return await get_book_info(book_name, author_name, api_key)

    # Run the asynchronous code
    tasks = [get_book_info_wrapper(row) for _, row in df.iterrows()]

    # Run the asynchronous code
    results = await asyncio.gather(*tasks)  # Corrected placement of await
    
    combined_book_info = pd.DataFrame()

    # Extract the actual results from the list
    for book_info in results:
        if book_info:
            authors = book_info.get('authors', [np.nan])
            publish_date = book_info.get('publishedDate', np.nan)
            description = book_info.get('description', np.nan)
            identifiers = book_info.get('industryIdentifiers', [])
            isbn = identifiers[0]['identifier'] if identifiers else np.nan
            page_count = book_info.get('pageCount', np.nan)
            categories = book_info.get('categories', [np.nan])
            average_rating = book_info.get('averageRating', np.nan)
            rating_count = book_info.get('ratingsCount', np.nan)
            language = book_info.get('language', np.nan)

            book_info_df = pd.DataFrame({
                'Title': [book_info.get('title', np.nan)],
                'Author(s)': [", ".join(map(str, authors))],
                'Publish Date': [publish_date],
                'Description': [description],
                'ISBN': [isbn],
                'Page Count': [page_count],
                'Categories': [", ".join(map(str, categories))],
                'Average Rating': [average_rating],
                'Rating Count': [rating_count],
                'Language': [language]
            })
            combined_book_info = pd.concat([combined_book_info, book_info_df])

    # Reset the index of the combined DataFrame
    combined_book_info = combined_book_info.reset_index(drop=True)
    combined_book_info = combined_book_info.rename(columns=lambda x: x.replace(' ', '_'))
   
from apps.api import api_key
result = await book_info_add(mybooksgr.sample(5), api_key)  # Use await when calling the async function
print(result)

None


In [6]:

import requests
import pandas as pd
import numpy as np
import aiohttp
import asyncio
import nest_asyncio
import time

async def get_book_info_async(session, book_name, author_name, api_key):
    base_url = 'https://www.googleapis.com/books/v1/volumes'
    params = {
        'q': f'intitle:{book_name}+inauthor:{author_name}',
        'key': api_key,
        'maxResults': 1,
        'fields': 'items(volumeInfo/title,volumeInfo/authors,volumeInfo/publishedDate,volumeInfo/description,volumeInfo/industryIdentifiers,volumeInfo/pageCount,volumeInfo/categories,volumeInfo/averageRating,volumeInfo/ratingsCount,volumeInfo/language)'
    }

    async with session.get(base_url, params=params) as response:
        if response.status == 200:
            data = await response.json()
            if 'items' in data and len(data['items']) > 0:
                return data['items'][0]['volumeInfo']
            print('request one done')
        else:
            print(f"Error {response.status}: {await response.text()}")
            return None

async def get_book_info(book_name, author_name, api_key):
    async with aiohttp.ClientSession() as session:
        return await get_book_info_async(session, book_name, author_name, api_key)

async def book_info_add(df, api_key):
    async def get_book_info_wrapper(row):
        book_name = row['Title']
        author_name = row['Author']
        await asyncio.sleep(1)
        return await get_book_info(book_name, author_name, api_key)

    # Run the asynchronous code
    tasks = [get_book_info_wrapper(row) for _, row in df.iterrows()]

    # Run the asynchronous code
    results = await asyncio.gather(*tasks)  # Corrected placement of await
    print('finishe, creating dfd')
    combined_book_info = pd.DataFrame()

    # Extract the actual results from the list
    for book_info in results:
        if book_info:
            authors = book_info.get('authors', [np.nan])
            publish_date = book_info.get('publishedDate', np.nan)
            description = book_info.get('description', np.nan)
            identifiers = book_info.get('industryIdentifiers', [])
            isbn = identifiers[0]['identifier'] if identifiers else np.nan
            page_count = book_info.get('pageCount', np.nan)
            categories = book_info.get('categories', [np.nan])
            average_rating = book_info.get('averageRating', np.nan)
            rating_count = book_info.get('ratingsCount', np.nan)
            language = book_info.get('language', np.nan)

            book_info_df = pd.DataFrame({
                'Title': [book_info.get('title', np.nan)],
                'Author(s)': [", ".join(map(str, authors))],
                'Publish Date': [publish_date],
                'Description': [description],
                'ISBN': [isbn],
                'Page Count': [page_count],
                'Categories': [", ".join(map(str, categories))],
                'Average Rating': [average_rating],
                'Rating Count': [rating_count],
                'Language': [language]
            })
            combined_book_info = pd.concat([combined_book_info, book_info_df])

    # Reset the index of the combined DataFrame
    combined_book_info = combined_book_info.reset_index(drop=True)
    combined_book_info = combined_book_info.rename(columns=lambda x: x.replace(' ', '_'))

    return combined_book_info


if __name__ == "__main__":
    asyncio.run(book_info_add(mybooksgr.sample(5), api_key))

finishe, creating dfd
